### Downloaded data 
https://www.kaggle.com/competitions/playground-series-s4e11/data
### Reading data 
train.csv from data folder 


In [1]:
### Install required libries 
! pip install numpy pandas matplotlib seaborn optuna scikit-learn catboost tqdm


In [2]:
import numpy as np
import pandas as pd 
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import copy
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from catboost import CatBoostClassifier, Pool
from tqdm import tqdm

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

data = pd.read_csv("../data/train.csv")

sample_percentage = 10 
sample_size = int(len(data) * (sample_percentage / 100))

data_sample, _ = train_test_split(data, train_size=sample_size, random_state=42, stratify=data['Depression'])

output_dir = "../data/raw"
os.makedirs(output_dir, exist_ok=True) 
output_file = os.path.join(output_dir, f"train_sample_{sample_percentage}p.csv")
data_sample.to_csv(output_file, index=False)

print(f"Sampled {sample_percentage}% of the data and saved to {output_file}")


Sampled 10% of the data and saved to ../data/raw/train_sample_10p.csv


In [4]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [11]:
num = train.select_dtypes(include= ['number']).columns.to_list()
cat = train.select_dtypes(include= ['category','object']).columns.to_list()
print(num)
print(cat)

['id', 'Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours', 'Financial Stress', 'Depression']
['Name', 'Gender', 'City', 'Working Professional or Student', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']


In [12]:
train[num].isna().sum()

id                         0
Age                        0
Academic Pressure     112803
Work Pressure          27918
CGPA                  112802
Study Satisfaction    112803
Job Satisfaction       27910
Work/Study Hours           0
Financial Stress           4
Depression                 0
dtype: int64